In [ ]:
import ee, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from featuretoolkit import src as ftk

In [3]:
from dotenv import load_dotenv

load_dotenv()
ee.Authenticate()
ee.Initialize(project=os.getenv('projectkey'))

TARGET_CRS="EPSG:32736"
TARGET_SCALE=90#meters
RECEPTIVE_FIELD=15#pixels^2

In [ ]:
gfld=pd.read_excel('database/19_10 Landslide database master.xlsx')
# courtesy of Professor Dave Petley

coolr=pd.read_csv('database/nasa_global_landslide_catalog_point.csv')
# courtesy of NASA Global Landslide Catalog @ https://maps.nccs.nasa.gov/arcgis/apps/MapAndAppGallery/index.html?appid=574f26408683485799d02e857e5d9521 (landslide reports points csv)

In [338]:
gfld.head()

,LandslideN,Country,Date,Day,Month,Year,Continent,Fatalities,Trigger,Location_M,...,Longitude,Location_R,Report 1,Source 1,Report 2,Source 2,Report 3,Source 3,Report 4,Source 4
0,1,Mozambique,2004-01-02,2,1,2004,E. Africa,6,unknown,Sofala,...,34.675341,Tsiquiri region Sofala province,NaN,http://allafrica.com/stories/200401050521.html,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Pakistan,2004-01-02,2,1,2004,S. Asia,3,construction,Shogran,...,73.462259,Shogran,NaN,http://paktribune.com/news/index.php?id=50368,NaN,NaN,NaN,NaN,NaN,NaN
2,3,China,2004-01-03,3,1,2004,E. Asia,16,unknown,Linfen,...,111.372850,Jingpo Village Linfen City Shanxi N. China,NaN,http://news.xinhuanet.com/english/2004-01/04/c...,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Indonesia,2004-01-04,4,1,2004,S.E. Asia,4,rainfall,Wanahayu,...,108.303381,Wanahayu village Maja subdistrict Majalengka...,NaN,http://www.antara.co.id/e_berita.asp?id=129215...,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Mexico,2004-01-11,11,1,2004,C. America,2,rainfall,Pichucalco,...,-93.188786,Pichucalco c.80 kms north of Tuxtla Gutierrez...,NaN,http://www.contracostatimes.com/mld/cctimes/ne...,NaN,NaN,NaN,NaN,NaN,NaN


In [339]:
coolr.head()

,OID,source_name,source_link,event_id,event_date,event_time,event_title,event_description,location_description,location_accuracy,...,event_import_id,latitude,longitude,country_name,country_code,admin_division_name,gazetteer_closest_point,gazetteer_distance,submitted_date,last_edited_date
7,-1,Maui Now,http://mauinow.com/2015/06/02/photos-motorists...,7014,6/1/2015 18:00:00,18:00,"Mile marker 20, Hana Highway",Debris from the landslide struck a single vehi...,"Mile marker 20, Hana Highway",1km,...,7014,20.8327,-156.1383,United States,US,Hawaii,Kailua,11.00,6/29/2015 20:05:53,NaN
9,-1,Jakarta Post,http://www.thejakartapost.com/news/2014/12/08/...,6437,1/4/2006 9:00:00,09:00,SIJERUK,The worst recorded landslide incident occurred...,SIJERUK,5km,...,6437,-6.9343,110.1969,Indonesia,ID,Jawa Tengah,Kendal,1.87,12/9/2014 15:57:33,NaN
10,-1,The Indian Express,http://indianexpress.com/article/india/india-o...,7972,8/28/2015 1:45:00,02:00,"Mizoram, India",A downpour was drenching the old building at 1...,"Mizoram, India",25km,...,7972,23.4495,92.8112,India,IN,Mizoram,Mizoram,9.74,6/30/2016 16:21:00,NaN
11,-1,Telemundo,http://www.telemundopr.com/telenoticias/puerto...,6708,5/18/2014 16:30:00,16:00,sector buena vista,Un deslizamiento de terrano pudiera haber pues...,sector buena vista,5km,...,6708,18.3806,-66.3319,Puerto Rico,US,Puerto Rico,Corozal,4.10,1/22/2015 16:32:57,NaN
12,-1,Oregon DOT,NaN,8755,12/28/2011 18:10:00,18:00,"OR 38, milepost 11",NaN,"OR 38, milepost 11",exact,...,8755,43.6567,-123.9163,United States,US,Oregon,Scottsburg,10.29,7/8/2016 16:00:34,NaN


In [342]:
coolr=ftk.process_coolr(coolr)
coolr

,time_start,time_end,spatial_uncertainty,latitude_center,longitude_center
0,1.136365e+12,1.136365e+12,5000,-6.934300,110.196900
1,1.440727e+12,1.440727e+12,25000,23.449500,92.811200
2,1.400386e+12,1.400386e+12,5000,18.380600,-66.331900
3,1.281053e+12,1.281053e+12,50000,34.141600,77.487100
4,1.300698e+12,1.300698e+12,5000,40.738800,-111.671200
...,...,...,...,...,...
2840,1.376737e+12,1.376737e+12,10000,24.762500,93.434100
2841,1.332403e+12,1.332403e+12,1000,39.054500,-94.647800
2842,1.550714e+12,1.550714e+12,500,36.505677,-83.022999
2843,1.747596e+12,1.747596e+12,500,35.463073,-83.013327


In [343]:
gfld=ftk.process_gfld(gfld)
gfld

,time_start,time_end,spatial_uncertainty,latitude_center,longitude_center
0,1.073174e+12,1.073261e+12,1271.402335,-6.925276,108.303381
1,1.073779e+12,1.073866e+12,14794.854925,17.552324,-93.188786
2,1.073779e+12,1.073866e+12,10062.257299,7.092358,80.886529
3,1.074038e+12,1.074125e+12,17313.149141,1.382049,110.117929
4,1.074125e+12,1.074211e+12,9063.467536,-22.585616,-43.327366
...,...,...,...,...,...
3596,1.513382e+12,1.513469e+12,4971.103785,11.560030,124.552919
3597,1.513382e+12,1.513469e+12,1164.719591,11.623736,124.426184
3598,1.513382e+12,1.513469e+12,1120.598696,-43.384466,-72.407090
3599,1.513901e+12,1.513987e+12,2336.392335,7.886669,125.170705


In [345]:
positives=ftk.concatenate(coolr,gfld)
positives

,event_id,time_start,time_end,spatial_uncertainty,latitude_center,longitude_center,region_id,fold_id
0,0,1.136365e+12,1.136365e+12,5000.000000,-6.934300,110.196900,828d8ffffffffff,818dbffffffffff
1,1,1.440727e+12,1.440727e+12,25000.000000,23.449500,92.811200,823cc7fffffffff,813cfffffffffff
2,2,1.400386e+12,1.400386e+12,5000.000000,18.380600,-66.331900,814cfffffffffff-OTHER,814cfffffffffff
3,3,1.281053e+12,1.281053e+12,50000.000000,34.141600,77.487100,823d27fffffffff,813d3ffffffffff
4,4,1.300698e+12,1.300698e+12,5000.000000,40.738800,-111.671200,822697fffffffff,8126bffffffffff
...,...,...,...,...,...,...,...,...
6441,6441,1.513382e+12,1.513469e+12,4971.103785,11.560030,124.552919,826867fffffffff,81687ffffffffff
6442,6442,1.513382e+12,1.513469e+12,1164.719591,11.623736,124.426184,826867fffffffff,81687ffffffffff
6443,6443,1.513382e+12,1.513469e+12,1120.598696,-43.384466,-72.407090,81cebffffffffff-OTHER,81cebffffffffff
6444,6444,1.513901e+12,1.513987e+12,2336.392335,7.886669,125.170705,82685ffffffffff,81687ffffffffff


In [ ]:
# thanks for your time